In [21]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def nba_calendar(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates={}".format(season)
    resp = download(url=url)
    if resp is not None:
        txt = json.loads(resp)['leagues'][0]['calendar']
        datenum = list(map(lambda x: x[:10].replace("-",""),txt))
        date = list(map(lambda x: x[:10],txt))

        year = list(map(lambda x: x[:4],txt))
        month = list(map(lambda x: x[5:7],txt))
        day = list(map(lambda x: x[8:10],txt))

        data = {"season": season,
                "datetime" : txt,
                "date" : date,
                "year": year,
                "month": month,
                "day": day,
                "dateURL": datenum

               }
        df = pd.DataFrame(data)
        df['url']="http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates="
        df['url']= df['url'] + df['dateURL']
        return df
    else:
        pass

nba_calendar(2018)

,season,datetime,date,year,month,day,dateURL,url
0,2018,2017-09-30T07:00Z,2017-09-30,2017,09,30,20170930,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20170930
1,2018,2017-10-01T07:00Z,2017-10-01,2017,10,01,20171001,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20171001
2,2018,2017-10-02T07:00Z,2017-10-02,2017,10,02,20171002,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20171002
3,2018,2017-10-03T07:00Z,2017-10-03,2017,10,03,20171003,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20171003
4,2018,2017-10-04T07:00Z,2017-10-04,2017,10,04,20171004,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20171004
...,...,...,...,...,...,...,...,...
223,2018,2018-05-28T07:00Z,2018-05-28,2018,05,28,20180528,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20180528
224,2018,2018-05-31T07:00Z,2018-05-31,2018,05,31,20180531,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20180531
225,2018,2018-06-03T07:00Z,2018-06-03,2018,06,03,20180603,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20180603
226,2018,2018-06-06T07:00Z,2018-06-06,2018,06,06,20180606,http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates=20180606


In [2]:
pd.set_option('display.max_colwidth',None)
schedule = {}
for x in range(2001,2022):
    full_schedule = nba_calendar(x)
    full_schedule.to_json(orient='records')
    schedule[x]=full_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('nba_calendar.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)    

In [19]:
def nba_schedule(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates={}".format(season)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
#     print(len(txt))
    txt = list(map(lambda x: x[:10].replace("-",""),txt))
    ev = pd.DataFrame()
    i=0
    for date in txt:
        print(f"Working on {season}: {i+1} of {len(txt)}, date: {txt[i]}")
        url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates={}&limit=1000".format(date)
        resp = download(url=url)
    
        if resp is not None:
            events_txt = json.loads(resp)

            events = events_txt['events']
            for event in events:
                bad_keys = ['linescores', 'statistics', 'leaders',  'records']
                for k in bad_keys:
                    if k in event['competitions'][0]['competitors'][0].keys():
                        del event['competitions'][0]['competitors'][0][k]
                    if k in event['competitions'][0]['competitors'][1].keys():
                        del event['competitions'][0]['competitors'][1][k]
                if 'links' in event['competitions'][0]['competitors'][0]['team'].keys():
                    del event['competitions'][0]['competitors'][0]['team']['links']
                if 'links' in event['competitions'][0]['competitors'][1]['team'].keys():
                    del event['competitions'][0]['competitors'][1]['team']['links']    
                if event['competitions'][0]['competitors'][0]['homeAway']=='home':
                    event['competitions'][0]['home'] = event['competitions'][0]['competitors'][0]['team']    
                else: 
                    event['competitions'][0]['away'] = event['competitions'][0]['competitors'][0]['team']
                if event['competitions'][0]['competitors'][1]['homeAway']=='away':
                    event['competitions'][0]['away'] = event['competitions'][0]['competitors'][1]['team']
                else: 
                    event['competitions'][0]['home'] = event['competitions'][0]['competitors'][1]['team']

                del_keys = ['competitors', 'broadcasts','geoBroadcasts', 'headlines']
                for k in del_keys:
                    if k in event['competitions'][0].keys():
                        del event['competitions'][0][k]

                ev = ev.append(pd.json_normalize(event['competitions'][0]))
            i+=1
            ev['season']=season
        else:
            i+=1
    return ev


In [20]:
schedule_table = pd.DataFrame()
schedule = {}
for x in range(2002,2022):
    year_schedule = nba_schedule(x)
    year_schedule.to_csv(f"nba/schedules/nba_games_info_{x}.csv")
    schedule_table = schedule_table.append(year_schedule)
    year_schedule.to_json(orient='records')
    schedule[x]=year_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}



schedule_table.to_csv('nba_games_info_2002_2021.csv')
    
# event['competitions'][0]

Working on 2002: 1 of 224, date: 20011009
Working on 2002: 2 of 224, date: 20011010
Working on 2002: 3 of 224, date: 20011011
Working on 2002: 4 of 224, date: 20011012
Working on 2002: 5 of 224, date: 20011013
Working on 2002: 6 of 224, date: 20011014
Working on 2002: 7 of 224, date: 20011015
Working on 2002: 8 of 224, date: 20011016
Working on 2002: 9 of 224, date: 20011017
Working on 2002: 10 of 224, date: 20011018
Working on 2002: 11 of 224, date: 20011019
Working on 2002: 12 of 224, date: 20011020
Working on 2002: 13 of 224, date: 20011021
Working on 2002: 14 of 224, date: 20011022
Working on 2002: 15 of 224, date: 20011023
Working on 2002: 16 of 224, date: 20011024
Working on 2002: 17 of 224, date: 20011025
Working on 2002: 18 of 224, date: 20011026
Working on 2002: 19 of 224, date: 20011030
Working on 2002: 20 of 224, date: 20011031
Working on 2002: 21 of 224, date: 20011101
Working on 2002: 22 of 224, date: 20011102
Working on 2002: 23 of 224, date: 20011103
Working on 2002: 24 

Working on 2002: 190 of 224, date: 20020427
Working on 2002: 191 of 224, date: 20020428
Working on 2002: 192 of 224, date: 20020429
Working on 2002: 193 of 224, date: 20020430
Working on 2002: 194 of 224, date: 20020501
Working on 2002: 195 of 224, date: 20020502
Working on 2002: 196 of 224, date: 20020503
Working on 2002: 197 of 224, date: 20020504
Working on 2002: 198 of 224, date: 20020505
Working on 2002: 199 of 224, date: 20020506
Working on 2002: 200 of 224, date: 20020507
Working on 2002: 201 of 224, date: 20020508
Working on 2002: 202 of 224, date: 20020509
Working on 2002: 203 of 224, date: 20020510
Working on 2002: 204 of 224, date: 20020511
Working on 2002: 205 of 224, date: 20020512
Working on 2002: 206 of 224, date: 20020513
Working on 2002: 207 of 224, date: 20020514
Working on 2002: 208 of 224, date: 20020515
Working on 2002: 209 of 224, date: 20020518
Working on 2002: 210 of 224, date: 20020519
Working on 2002: 211 of 224, date: 20020520
Working on 2002: 212 of 224, dat

Working on 2003: 147 of 228, date: 20030309
Working on 2003: 148 of 228, date: 20030310
Working on 2003: 149 of 228, date: 20030311
Working on 2003: 150 of 228, date: 20030312
Working on 2003: 151 of 228, date: 20030313
Working on 2003: 152 of 228, date: 20030314
Working on 2003: 153 of 228, date: 20030315
Working on 2003: 154 of 228, date: 20030316
Working on 2003: 155 of 228, date: 20030317
Working on 2003: 156 of 228, date: 20030318
Working on 2003: 157 of 228, date: 20030319
Working on 2003: 158 of 228, date: 20030320
Working on 2003: 159 of 228, date: 20030321
Working on 2003: 160 of 228, date: 20030322
Working on 2003: 161 of 228, date: 20030323
Working on 2003: 162 of 228, date: 20030324
Working on 2003: 163 of 228, date: 20030325
Working on 2003: 164 of 228, date: 20030326
Working on 2003: 165 of 228, date: 20030327
Working on 2003: 166 of 228, date: 20030328
Working on 2003: 167 of 228, date: 20030329
Working on 2003: 168 of 228, date: 20030330
Working on 2003: 169 of 228, dat

Working on 2004: 100 of 234, date: 20040117
Working on 2004: 101 of 234, date: 20040118
Working on 2004: 102 of 234, date: 20040119
Working on 2004: 103 of 234, date: 20040120
Working on 2004: 104 of 234, date: 20040121
Working on 2004: 105 of 234, date: 20040122
Working on 2004: 106 of 234, date: 20040123
Working on 2004: 107 of 234, date: 20040124
Working on 2004: 108 of 234, date: 20040125
Working on 2004: 109 of 234, date: 20040126
Working on 2004: 110 of 234, date: 20040127
Working on 2004: 111 of 234, date: 20040128
Working on 2004: 112 of 234, date: 20040129
Working on 2004: 113 of 234, date: 20040130
Working on 2004: 114 of 234, date: 20040131
Working on 2004: 115 of 234, date: 20040201
Working on 2004: 116 of 234, date: 20040202
Working on 2004: 117 of 234, date: 20040203
Working on 2004: 118 of 234, date: 20040204
Working on 2004: 119 of 234, date: 20040205
Working on 2004: 120 of 234, date: 20040206
Working on 2004: 121 of 234, date: 20040207
Working on 2004: 122 of 234, dat

Working on 2005: 54 of 231, date: 20041205
Working on 2005: 55 of 231, date: 20041206
Working on 2005: 56 of 231, date: 20041207
Working on 2005: 57 of 231, date: 20041208
Working on 2005: 58 of 231, date: 20041209
Working on 2005: 59 of 231, date: 20041210
Working on 2005: 60 of 231, date: 20041211
Working on 2005: 61 of 231, date: 20041212
Working on 2005: 62 of 231, date: 20041213
Working on 2005: 63 of 231, date: 20041214
Working on 2005: 64 of 231, date: 20041215
Working on 2005: 65 of 231, date: 20041216
Working on 2005: 66 of 231, date: 20041217
Working on 2005: 67 of 231, date: 20041218
Working on 2005: 68 of 231, date: 20041219
Working on 2005: 69 of 231, date: 20041220
Working on 2005: 70 of 231, date: 20041221
Working on 2005: 71 of 231, date: 20041222
Working on 2005: 72 of 231, date: 20041223
Working on 2005: 73 of 231, date: 20041225
Working on 2005: 74 of 231, date: 20041226
Working on 2005: 75 of 231, date: 20041227
Working on 2005: 76 of 231, date: 20041228
Working on 

...

Working on 2017: 182 of 226, date: 20170410
Working on 2017: 183 of 226, date: 20170411
Working on 2017: 184 of 226, date: 20170412
Working on 2017: 185 of 226, date: 20170415
Working on 2017: 186 of 226, date: 20170416
Working on 2017: 187 of 226, date: 20170417
Working on 2017: 188 of 226, date: 20170418
Working on 2017: 189 of 226, date: 20170419
Working on 2017: 190 of 226, date: 20170420
Working on 2017: 191 of 226, date: 20170421
Working on 2017: 192 of 226, date: 20170422
Working on 2017: 193 of 226, date: 20170423
Working on 2017: 194 of 226, date: 20170424
Working on 2017: 195 of 226, date: 20170425
Working on 2017: 196 of 226, date: 20170426
Working on 2017: 197 of 226, date: 20170427
Working on 2017: 198 of 226, date: 20170428
Working on 2017: 199 of 226, date: 20170430
Working on 2017: 200 of 226, date: 20170501
Working on 2017: 201 of 226, date: 20170502
Working on 2017: 202 of 226, date: 20170503
Working on 2017: 203 of 226, date: 20170504
Working on 2017: 204 of 226, dat

Working on 2018: 145 of 228, date: 20180302
Working on 2018: 146 of 228, date: 20180303
Working on 2018: 147 of 228, date: 20180304
Working on 2018: 148 of 228, date: 20180305
Working on 2018: 149 of 228, date: 20180306
Working on 2018: 150 of 228, date: 20180307
Working on 2018: 151 of 228, date: 20180308
Working on 2018: 152 of 228, date: 20180309
Working on 2018: 153 of 228, date: 20180310
Working on 2018: 154 of 228, date: 20180311
Working on 2018: 155 of 228, date: 20180312
Working on 2018: 156 of 228, date: 20180313
Working on 2018: 157 of 228, date: 20180314
Working on 2018: 158 of 228, date: 20180315
Working on 2018: 159 of 228, date: 20180316
Working on 2018: 160 of 228, date: 20180317
Working on 2018: 161 of 228, date: 20180318
Working on 2018: 162 of 228, date: 20180319
Working on 2018: 163 of 228, date: 20180320
Working on 2018: 164 of 228, date: 20180321
Working on 2018: 165 of 228, date: 20180322
Working on 2018: 166 of 228, date: 20180323
Working on 2018: 167 of 228, dat

Working on 2019: 106 of 230, date: 20190116
Working on 2019: 107 of 230, date: 20190117
Working on 2019: 108 of 230, date: 20190118
Working on 2019: 109 of 230, date: 20190119
Working on 2019: 110 of 230, date: 20190120
Working on 2019: 111 of 230, date: 20190121
Working on 2019: 112 of 230, date: 20190122
Working on 2019: 113 of 230, date: 20190123
Working on 2019: 114 of 230, date: 20190124
Working on 2019: 115 of 230, date: 20190125
Working on 2019: 116 of 230, date: 20190126
Working on 2019: 117 of 230, date: 20190127
Working on 2019: 118 of 230, date: 20190128
Working on 2019: 119 of 230, date: 20190129
Working on 2019: 120 of 230, date: 20190130
Working on 2019: 121 of 230, date: 20190131
Working on 2019: 122 of 230, date: 20190201
Working on 2019: 123 of 230, date: 20190202
Working on 2019: 124 of 230, date: 20190203
Working on 2019: 125 of 230, date: 20190204
Working on 2019: 126 of 230, date: 20190205
Working on 2019: 127 of 230, date: 20190206
Working on 2019: 128 of 230, dat

Working on 2020: 55 of 214, date: 20191130
Working on 2020: 56 of 214, date: 20191201
Working on 2020: 57 of 214, date: 20191202
Working on 2020: 58 of 214, date: 20191203
Working on 2020: 59 of 214, date: 20191204
Working on 2020: 60 of 214, date: 20191205
Working on 2020: 61 of 214, date: 20191206
Working on 2020: 62 of 214, date: 20191207
Working on 2020: 63 of 214, date: 20191208
Working on 2020: 64 of 214, date: 20191209
Working on 2020: 65 of 214, date: 20191210
Working on 2020: 66 of 214, date: 20191211
Working on 2020: 67 of 214, date: 20191212
Working on 2020: 68 of 214, date: 20191213
Working on 2020: 69 of 214, date: 20191214
Working on 2020: 70 of 214, date: 20191215
Working on 2020: 71 of 214, date: 20191216
Working on 2020: 72 of 214, date: 20191217
Working on 2020: 73 of 214, date: 20191218
Working on 2020: 74 of 214, date: 20191219
Working on 2020: 75 of 214, date: 20191220
Working on 2020: 76 of 214, date: 20191221
Working on 2020: 77 of 214, date: 20191222
Working on 

Working on 2021: 30 of 150, date: 20210112
Working on 2021: 31 of 150, date: 20210113
Working on 2021: 32 of 150, date: 20210114
Working on 2021: 33 of 150, date: 20210115
Working on 2021: 34 of 150, date: 20210116
Working on 2021: 35 of 150, date: 20210117
Working on 2021: 36 of 150, date: 20210118
Working on 2021: 37 of 150, date: 20210119
Working on 2021: 38 of 150, date: 20210120
Working on 2021: 39 of 150, date: 20210121
Working on 2021: 40 of 150, date: 20210122
Working on 2021: 41 of 150, date: 20210123
Working on 2021: 42 of 150, date: 20210124
Working on 2021: 43 of 150, date: 20210125
Working on 2021: 44 of 150, date: 20210126
Working on 2021: 45 of 150, date: 20210127
Working on 2021: 46 of 150, date: 20210128
Working on 2021: 47 of 150, date: 20210129
Working on 2021: 48 of 150, date: 20210130
Working on 2021: 49 of 150, date: 20210131
Working on 2021: 50 of 150, date: 20210201
Working on 2021: 51 of 150, date: 20210202
Working on 2021: 52 of 150, date: 20210203
Working on 